In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, BlackLittermanModel
import yfinance as yf
import pypfopt

In [3]:
data = {}
start_date = '2020-01-01'
end_date = '2020-12-31'

In [4]:
data['VTI'] = yf.download('VTI', period='max')
data['VTV'] = yf.download('VTV', period='max')
data['VOE'] = yf.download('VOE', period='max')
data['VBR'] = yf.download('VBR', period='max')
data['GBIL'] = yf.download('GBIL', period='max')
data['JPST'] = yf.download('JPST', period='max')
data['VNQ'] = yf.download('VNQ', period='max')
data['VPU'] = yf.download('VPU', period='max')
#data['BITO'] = yf.download('BITO', period='max')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
for ticker in data:
    data[ticker].to_csv(ticker + '.csv')

for ticker in data:
    data[ticker]['percentageChange'] = data[ticker]['Close'].pct_change()*10
    data[ticker]['percentageChange'] = data[ticker]['percentageChange'].fillna(0)

In [6]:
onlyClosePrices = pd.DataFrame()
for ticker in data:
    onlyClosePrices[ticker] = data[ticker]['Close']

In [7]:
S = pypfopt.risk_models.CovarianceShrinkage(onlyClosePrices).ledoit_wolf()

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import time #helper libraries

In [9]:
scalar = MinMaxScaler(feature_range=(-1, 1))

In [10]:
data['VTI'] = scalar.fit_transform(data['VTI'].dropna())
data['VTV'] = scalar.fit_transform(data['VTV'].dropna())
data['VOE'] = scalar.fit_transform(data['VOE'].dropna())
data['VBR'] = scalar.fit_transform(data['VBR'].dropna())
data['GBIL'] = scalar.fit_transform(data['GBIL'].dropna())
data['JPST'] = scalar.fit_transform(data['JPST'].dropna())
data['VNQ'] = scalar.fit_transform(data['VNQ'].dropna())
data['VPU'] = scalar.fit_transform(data['VPU'].dropna())
#data['BITO'] = scalar.fit_transform(data['BITO'].dropna())

In [11]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), -2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [12]:
trainingData = {}
testData = {}

In [13]:
for ticker in data:
    trainingData[ticker], testData[ticker] = train_test_split(data[ticker], test_size=0.05, shuffle=False)

In [14]:
trainX, trainY = {}, {}
testX, testY = {}, {}
timeStep = 100
for ticker in data:
    trainX[ticker], trainY[ticker] = create_dataset(trainingData[ticker], timeStep)
    testX[ticker], testY[ticker] = create_dataset(testData[ticker], timeStep)

In [15]:
trainX['VTI'].shape

(4911, 100)

In [16]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [17]:
for ticker in data:
    model.fit(trainX[ticker], trainY[ticker], epochs=100, batch_size=5, verbose=1, validation_data=(testX[ticker], testY[ticker]))
    model.save(ticker + '.h5')
    model.save_weights(ticker + '_weights.h5')

Epoch 1/100
983/983 [==============================] - ETA: 0s - loss: 0.0117 - root_mean_squared_error: 0.1084

In [33]:
model = load_model('VTI.h5')

In [34]:
viewDict = {}

In [35]:
models = {}
for ticker in data:
    print(ticker)
    models[ticker] = load_model(ticker + '.h5')
    viewDict[ticker] = model.predict(testX[ticker])

VTI
VTV
VOE
VBR
GBIL
JPST
VNQ
VPU


In [23]:
trainX['VTI'].shape

(25214, 100)

In [36]:
viewDict['VTI']

array([[0.00237771],
       [0.00232566],
       [0.00225674],
       [0.00217099],
       [0.00207224],
       [0.00197094],
       [0.00187956],
       [0.00180816],
       [0.00175917],
       [0.00172914],
       [0.00171137],
       [0.00169976],
       [0.00169315],
       [0.00169708],
       [0.00171971],
       [0.00176058],
       [0.00181364],
       [0.00187481],
       [0.0019374 ],
       [0.00199279],
       [0.0020324 ],
       [0.00204906],
       [0.00203762],
       [0.00199648],
       [0.00192725],
       [0.00183542],
       [0.0017294 ],
       [0.00161672],
       [0.00150534],
       [0.00140343],
       [0.00132022],
       [0.00127065],
       [0.00126   ],
       [0.00128591],
       [0.00135105],
       [0.00144914],
       [0.00156544],
       [0.00168337],
       [0.00179084],
       [0.00187862],
       [0.00194252],
       [0.00198496],
       [0.00200765],
       [0.00201283],
       [0.0020057 ],
       [0.00199096],
       [0.00197482],
       [0.001

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypfopt
import yfinance as yf
import pandas_datareader as pdr
import seaborn as sns
import random

In [38]:
netAssets = {}
for ticker in data:
    netAssets[ticker] = yf.Ticker(ticker).info['totalAssets']

In [87]:
netAssets

{'VTI': 1200912072704,
 'VTV': 145279369216,
 'VOE': 28818311168,
 'VBR': 47090102272,
 'GBIL': 2155137536,
 'JPST': 18943338496,
 'VNQ': 82814885888,
 'VPU': 7799145984}

In [97]:
csvAssets = pd.read_csv('assets.csv', index_col=0)

In [103]:
csvAssets['0']['VTI']

1200912072704

In [39]:
delta = pypfopt.black_litterman.market_implied_risk_aversion(pd.Series(netAssets), risk_free_rate=0.05796)

In [40]:
marketPrior = pypfopt.black_litterman.market_implied_prior_returns(netAssets, delta, S)

In [55]:
viewDict = {}
confidence = []
for ticker in data:
    pred = models[ticker].predict(testX[ticker])
    viewDict[ticker] = (pred[-1,0] - testX[ticker][-2, 0])/testX[ticker][-2, 0]
    confidence.append(1-model.evaluate(testX[ticker], testY[ticker])[1])

26/26 [==============================] - 1s 31ms/step - loss: 0.2825 - root_mean_squared_error: 0.5315


In [47]:
viewDict

{'VTI': -1.009140012611804,
 'VTV': -1.0570494940347552,
 'VOE': -1.0062323777158684,
 'VBR': -1.0360466401759705,
 'GBIL': -0.991965489799386,
 'JPST': -0.9895737999220977,
 'VNQ': -1.0429912344853445,
 'VPU': -0.9587023928597205}

In [56]:
bl = pypfopt.black_litterman.BlackLittermanModel(S, pi=marketPrior, absolute_views=viewDict, rho=delta)

In [57]:
bl_confi = pypfopt.BlackLittermanModel(S, absolute_views=viewDict, omega="idzorek", rho=delta, view_confidences=confidence)

/home/rootdaenib/.local/lib/python3.8/site-packages/pypfopt/black_litterman.py:257: UserWarning: Running Black-Litterman with no prior.
  warnings.warn("Running Black-Litterman with no prior.")


In [58]:
bl_return_confi = bl_confi.bl_returns()
bl_return_confi.name = 'BL Returns with Confidence'

In [59]:
S_bl_confi = bl_confi.bl_cov()

In [60]:
ef = pypfopt.EfficientFrontier(bl_return_confi, S_bl_confi, weight_bounds=(0, 0.2))

In [62]:
#ef.add_objective('Sharpe', 'max')
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)

Expected annual return: -69.1%
Annual volatility: 10.1%
Sharpe Ratio: -7.03


(-0.6912720294501844, 0.10121955012707227, -7.027022235894595)

In [75]:
import json
json.dumps(weights)

'{"VTI": 0.1520113679582948, "VTV": 0.2, "VOE": 0.0479886320417051, "VBR": 0.0, "GBIL": 0.2, "JPST": 0.2, "VNQ": 0.0, "VPU": 0.2}'

In [63]:
wt_min_vola = pd.DataFrame([weights], columns=weights.keys()).T*100
print ('Weights in Percentage ********************')
print(wt_min_vola.round(4))

Weights in Percentage ********************
            0
VTI   15.2011
VTV   20.0000
VOE    4.7989
VBR    0.0000
GBIL  20.0000
JPST  20.0000
VNQ    0.0000
VPU   20.0000


In [76]:
data

{'VTI': array([[-0.79286869, -0.79647497, -0.76099121, -0.7900798 , -0.88305172,
         -0.95008581],
        [-0.78915547, -0.79733473, -0.75972162, -0.79347254, -0.88528888,
         -0.98718815],
        [-0.78596588, -0.7949465 , -0.76042694, -0.79199121, -0.88431199,
         -0.91651026],
        ...,
        [ 0.65666954,  0.65857861,  0.63351684,  0.6371195 ,  0.65678366,
         -0.85456901],
        [ 0.63124825,  0.66698509,  0.63859506,  0.67706782,  0.69533414,
         -0.85976939],
        [ 0.64724361,  0.65179597,  0.65082061,  0.64514737,  0.66453062,
         -0.87323364]]),
 'VTV': array([[-0.65740739, -0.67358272, -0.57997653, -0.66379946, -0.84516763,
         -0.99791637],
        [-0.65789476, -0.66851823, -0.57997653, -0.66117802, -0.84365417,
         -0.99963328],
        [-0.65627029, -0.67243913, -0.57763001, -0.66134183, -0.8437487 ,
         -0.99948326],
        ...,
        [ 0.88986356,  0.88400578,  0.8665624 ,  0.87154917,  0.87984836,
         -0